# Medical Imaging Workshop - Notebook 1: Dataset Setup & Visualization
Load local dataset in Fiftyone

In [ ]:
%pip install fiftyone eyepop tqdm

In [ ]:
import fiftyone as fo
import os
import json

In [ ]:
accountUUID = "60107c508dde44b19c3e7d1f30a1e2c0"
datasetUUID = "0686c4296f957d3980005a829876d16e"
apikey = "AAE_w6lCcrCa27chNAbZO-WdZ0FBQUFBQmwyUFk5bmtLZnJBQ2RFVWVDbzU1MnkwTUMzYXhQWjA4a0ZEczFKWWdONjdRS0NGWUZ5aF90aXVQZ3FrcWdkZWwwUEx6Q0luM0F3b3ItMjdqRmhUQkxyTWVvSndFLWRCUENjZGNlanZhbGhRTDdtV289"
cache_directory = "./.cache/voxel51/" + accountUUID + "/" + datasetUUID

In [ ]:
async def run():
    print("Images directory:", os.listdir(os.path.join(cache_directory, "data"))[:5])
    print("Annotations file exists:", os.path.exists(os.path.join(cache_directory, "annotations", "annotations.json")))
    with open(os.path.join(cache_directory, "annotations", "annotations.json")) as f:
        coco = json.load(f)
        print("Total images in COCO:", len(coco["images"]))
        print("Total annotations:", len(coco["annotations"]))

    print("Loading dataset into FiftyOne...")    
    dataset = fo.Dataset.from_dir(
        dataset_dir=os.path.expanduser(cache_directory),
        dataset_type=fo.types.COCODetectionDataset,
        labels_path="annotations/annotations.json",
    )
    session = fo.launch_app(dataset, browser=True)
    
await run()

In [ ]:
import webbrowser

webbrowser.open("http://localhost:5151/")